In [3]:
import pandas as pd
import plotly.express as px

In [14]:
# 1. Load the data (skipping the LinkedIn header notes)
df_conn = pd.read_csv('data/Connections.csv', skiprows=3)
df_inv = pd.read_csv('data/Invitations.csv')
df_search = pd.read_csv('data/SearchQueries.csv', on_bad_lines='skip')


In [15]:
# 2. Calculate absolute counts of missing values
missing_counts = df_conn.isnull().sum()
print(missing_counts)

First Name          6
Last Name           6
URL                 6
Email Address    1010
Company            39
Position           39
Connected On        0
dtype: int64


In [16]:
# 2. Fill the 1010 missing IDs/Emails
# We give them a placeholder so we can still count them in 'Total Network'
df_conn['Email Address'] = df_conn['Email Address'].fillna('Private_Profile')

# 3. Fill the 39 missing Company/Position
df_conn['Company'] = df_conn['Company'].fillna('Not Listed')
df_conn['Position'] = df_conn['Position'].fillna('Professional')

In [17]:
# Remove rows where 'First Name' is missing
# subset=['First Name'] ensures we ONLY drop rows missing the name, not other columns
df_conn = df_conn.dropna(subset=['First Name'])

In [18]:
# Final Verification
print("--- Remaining Nulls ---")
print(df_conn.isnull().sum())

--- Remaining Nulls ---
First Name       0
Last Name        0
URL              0
Email Address    0
Company          0
Position         0
Connected On     0
dtype: int64


In [19]:
# 1. Standardize the Date
df_conn['Connected On'] = pd.to_datetime(df_conn['Connected On'], dayfirst=True)

# 2. Create a Month-Year column for grouping
df_conn['Month_Year'] = df_conn['Connected On'].dt.to_period('M').astype(str)

In [20]:
# 3. Calculate Velocity
velocity = df_conn.groupby('Month_Year').size().reset_index(name='New_Connections')

# 4. Plot the "Heartbeat" of your career
fig = px.line(velocity, x='Month_Year', y='New_Connections', 
              title="Networking Velocity: Identifying Periods of Network Decay",
              line_shape='spline', render_mode='svg')
fig.show()

In [21]:
# Calculate Company Concentration
company_counts = df_conn['Company'].value_counts().reset_index()
company_counts.columns = ['Company', 'Count']
company_counts['Percentage'] = (company_counts['Count'] / len(df_conn)) * 100

# Visualize Saturation using a Treemap (Best for showing 'Over-Indexing')
fig_sat = px.treemap(company_counts.head(20), path=['Company'], values='Count',
                     color='Percentage', color_continuous_scale='RdYlGn_r',
                     title="Company Saturation: Identifying 'Fragile' Clusters")
fig_sat.show()

In [22]:
import pandas as pd

# 1. Create Time Buckets
df_conn['Connected On'] = pd.to_datetime(df_conn['Connected On'])
df_conn['Year'] = df_conn['Connected On'].dt.year
df_conn['Month_Year'] = df_conn['Connected On'].dt.to_period('M').astype(str)

# 2. Map Seniority (The Storytelling Engine)
def map_seniority(title):
    t = str(title).lower()
    if any(word in t for word in ['director', 'vp', 'chief', 'head', 'founder', 'ceo', 'cto']):
        return 'Leadership'
    elif any(word in t for word in ['senior', 'sr', 'lead', 'principal', 'manager']):
        return 'Senior/Mid'
    elif any(word in t for word in ['intern', 'junior', 'associate', 'grad', 'student']):
        return 'Junior/Entry'
    else:
        return 'Individual Contributor'

df_conn['Seniority_Level'] = df_conn['Position'].apply(map_seniority)

# 3. Check your work
print("Top 5 Years of Networking Activity:")
print(df_conn['Year'].value_counts().head())
print("\nSeniority Distribution:")
print(df_conn['Seniority_Level'].value_counts())

Top 5 Years of Networking Activity:
Year
2024    568
2025    313
2019     36
2026     27
2020     27
Name: count, dtype: int64

Seniority Distribution:
Seniority_Level
Individual Contributor    612
Senior/Mid                270
Junior/Entry              112
Leadership                 35
Name: count, dtype: int64


In [23]:
def categorize_role(position):
    p = str(position).lower()
    if any(x in p for x in ['engineer', 'developer', 'software', 'tech', 'data']):
        return 'Engineering/Tech'
    elif any(x in p for x in ['recruiter', 'talent', 'hr', 'acquisition']):
        return 'Human Resources'
    elif any(x in p for x in ['product', 'design', 'ux', 'ui']):
        return 'Product/Design'
    elif any(x in p for x in ['director', 'vp', 'founder', 'chief', 'ceo', 'head']):
        return 'Leadership'
    elif any(x in p for x in ['sales', 'marketing', 'growth', 'business dev']):
        return 'Sales/Marketing'
    else:
        return 'Other/General'

df_conn['Industry_Group'] = df_conn['Position'].apply(categorize_role)

In [32]:

# 1. Prepare and Sort
df_conn['Connected On'] = pd.to_datetime(df_conn['Connected On'])
df_conn = df_conn.sort_values('Connected On')

# 2. Resample & Fill Holes (The Secret to a "Smooth" Chart)
# This creates a grid of every Month vs every Industry Group
pivot_df = df_conn.groupby([df_conn['Connected On'].dt.to_period('M'), 'Industry_Group']).size().unstack(fill_value=0)

# 3. Cumulative Growth
# We calculate the running total so the chart always moves upward
cumulative_df = pivot_df.cumsum()
cumulative_df.index = cumulative_df.index.to_timestamp()

# 4. Flatten for Plotly
plot_df = cumulative_df.reset_index().melt(id_vars='Connected On', var_name='Industry', value_name='Total')

# 5. Create the Polished Visualization
fig = px.area(
    plot_df, 
    x="Connected On", 
    y="Total", 
    color="Industry",
    title="<b>The Career Mirror: Professional Evolution</b><br><sup>Mapping the transition of 10,000+ connections over time</sup>",
    labels={"Total": "Cumulative Connections", "Connected On": ""},
    color_discrete_sequence=px.colors.qualitative.Safe, # High-contrast, clean colors
    template="plotly_white" # Cleaner background for presentations
)

# 6. Professional Layout Tweaks
fig.update_layout(
    hovermode="x unified",  # Shows all industry values in one tooltip
    font_family="Arial",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=0, r=20, t=100, b=0)
)

fig.update_xaxes(
    showgrid=False, 
    rangeslider_visible=True, # Allows you to zoom into specific "Career Sprints"
    type='date'
)

fig.update_yaxes(showgrid=True, gridcolor='rgba(230,230,230,0.5)')

fig.show()


In [36]:
import plotly.graph_objects as go

# Using the resilience_score calculated previously
fig_gauge = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = resilience_score,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "<b>Network Resilience Index</b><br><span style='font-size:0.8em;color:gray'>Industry Diversity Score</span>", 'font': {'size': 20}},
    number = {'font': {'size': 50, 'family': "Arial"}, 'valueformat': ".2f"},
    gauge = {
        'axis': {'range': [0, 1], 'tickwidth': 1, 'tickcolor': "darkblue"},
        'bar': {'color': "#636EFA"}, # Sleek Blue
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "#D3D3D3",
        'steps': [
            {'range': [0, 0.4], 'color': '#F3F4F7'},
            {'range': [0.4, 0.7], 'color': '#E5E7EB'},
            {'range': [0.7, 1], 'color': '#D1D5DB'}],
        'threshold': {
            'line': {'color': "#EF553B", 'width': 4},
            'thickness': 0.75,
            'value': 0.8} # Your personal goal
    }
))

fig_gauge.update_layout(
    paper_bgcolor = "#f9f9f9", # Soft paper background
    font = {'color': "#2D3436", 'family': "Arial"},
    height = 400
)
fig_gauge.show()

In [37]:
# 1. Create a binary column: 1 if Email exists, 0 if Private
df_conn['Is_Direct'] = df_conn['Email Address'].apply(lambda x: 0 if x == 'Private_Profile' else 1)

# 2. Group by Quarter to see the trend
direct_trend = df_conn.groupby(df_conn['Connected On'].dt.to_period('Q'))['Is_Direct'].mean().reset_index()
direct_trend['Connected On'] = direct_trend['Connected On'].dt.to_timestamp()

# 3. Plot the "Directness" Ratio
fig_ratio = px.line(direct_trend, x='Connected On', y='Is_Direct',
              title="Networking Quality Trend: % of Connections with Direct Contact Info",
              labels={'Is_Direct': '% of Direct Contacts'},
              markers=True, template="plotly_white")

# Format y-axis as percentage
fig_ratio.update_layout(yaxis_tickformat='.0%')
fig_ratio.show()

In [38]:
df_conn.head()

,First Name,Last Name,URL,Email Address,Company,Position,Connected On,Month_Year,Year,Seniority_Level,Industry_Group,Is_Direct
1034,Neha,Hiray,https://www.linkedin.com/in/neha-hiray-38395a17a,Private_Profile,SMS-Magic,Software Developer,2019-07-27,2019-07,2019,Individual Contributor,Engineering/Tech,0
1030,Gautami,Dhokte,https://www.linkedin.com/in/gautami-dhokte-951...,Private_Profile,Winjit Technologies Pvt Ltd,Technical Lead,2019-07-27,2019-07,2019,Senior/Mid,Engineering/Tech,0
1033,Saloni,Lunawat,https://www.linkedin.com/in/saloni-lunawat-aa9...,Private_Profile,WhiteOak Capital,Assistant Digital Product Manager,2019-07-27,2019-07,2019,Senior/Mid,Product/Design,0
1032,Aditi,Bhadane,https://www.linkedin.com/in/aditi-bhadane-1596...,Private_Profile,Deloitte India,Assistant Manager,2019-07-27,2019-07,2019,Senior/Mid,Other/General,0
1031,Reema,Patel,https://www.linkedin.com/in/reema-patel-b4a11a157,Private_Profile,Omni Logic Solutions,Techno- Functional Analyst,2019-07-27,2019-07,2019,Individual Contributor,Engineering/Tech,0


In [48]:
from datetime import datetime

# 1. Define your "Pivot Date" and "Legacy Industries"
five_years_ago = datetime.now().year - 5
legacy_industries = ['Sales/Marketing', 'Human Resources', 'Other/General'] # Adjust based on your history

# 2. Flag the Clutter
def flag_legacy(row):
    is_old = row['Connected On'].year <= five_years_ago
    is_legacy_industry = row['Industry_Group'] in legacy_industries
    
    if is_old and is_legacy_industry:
        return 'Legacy Clutter'
    elif is_old:
        return 'Old but Relevant'
    else:
        return 'Modern Network'

df_conn['Network_Status'] = df_conn.apply(flag_legacy, axis=1)

# 3. Calculate the "Dead Weight" Percentage
clutter_count = len(df_conn[df_conn['Network_Status'] == 'Legacy Clutter'])
clutter_percent = (clutter_count / len(df_conn)) * 100

print(f"⚠️ Legacy Clutter: {clutter_count} connections ({clutter_percent:.1f}% of your network)")

⚠️ Legacy Clutter: 32 connections (3.1% of your network)


In [49]:
import plotly.express as px

fig_clutter = px.pie(
    df_conn, 
    names='Network_Status', 
    hole=0.6,
    title="<b>Network Composition: Modern vs. Legacy</b>",
    color_discrete_map={
        'Modern Network': '#00D1B2',    # Fresh Teal
        'Old but Relevant': '#636EFA',  # Reliable Blue
        'Legacy Clutter': '#EF553B'      # Warning Red
    },
    template="plotly_white"
)

fig_clutter.update_layout(showlegend=True)
fig_clutter.show()